
## Giulia Mancini


##### a) Load CSV files ‘customers.csv’, ‘order_items.csv’, ‘orders.csv’, and ‘products.csv’ and then populate (fill) an SQLite database with that data in tables for each file


Import packages 


In [ ]:
import sqlite3
import pandas as pd


We will load data from `customers.csv`, `order_items.csv`, `orders.csv`, and `products.csv` into an SQLite database.


In [ ]:
file_paths = {
    "customers": "/Users/giuliamancini/Desktop/customers.csv",
    "order_items": "/Users/giuliamancini/Desktop/order_items.csv",
    "orders": "/Users/giuliamancini/Desktop/orders.csv",
    "products": "/Users/giuliamancini/Desktop/products.csv"
}

I used this path because all my docs are in my Desktop.

Load CSVs into DataFrames

In [ ]:
dfs = {name: pd.read_csv(path) for name, path in file_paths.items()}

Connect to SQLite database

In [ ]:
db_path = "/Users/giuliamancini/Desktop/store_database.sqlite"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

Define table creation queries

In [ ]:
table_schemas = {
    "customers": """
        CREATE TABLE IF NOT EXISTS customers (
            customer_id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            email TEXT
        )
    """,
    "orders": """
        CREATE TABLE IF NOT EXISTS orders (
            order_id INTEGER PRIMARY KEY,
            customer_id INTEGER,
            order_date TEXT,
            FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
        )
    """,
    "products": """
        CREATE TABLE IF NOT EXISTS products (
            product_id INTEGER PRIMARY KEY,
            product_name TEXT NOT NULL,
            category TEXT,
            price REAL
        )
    """,
    "order_items": """
        CREATE TABLE IF NOT EXISTS order_items (
            id INTEGER PRIMARY KEY,
            order_id INTEGER,
            product_id INTEGER,
            quantity INTEGER,
            FOREIGN KEY (order_id) REFERENCES orders(order_id),
            FOREIGN KEY (product_id) REFERENCES products(product_id)
        )
    """
}

Create tables

In [ ]:
for table, schema in table_schemas.items():
    cursor.execute(schema)

Insert data into tables

In [ ]:
for table, df in dfs.items():
    df.to_sql(table, conn, if_exists="replace", index=False)

Verify tables

In [ ]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print("Tables in database:", cursor.fetchall())

Tables in database: [('customers',), ('order_items',), ('orders',), ('products',)]


Check if tables contain data by querying the first 5 rows.

In [ ]:
conn = sqlite3.connect(db_path)

# Query and display first 5 rows of each table
tables = ["customers", "orders", "products", "order_items"]
for table in tables:
    df = pd.read_sql_query(f"SELECT * FROM {table} LIMIT 5;", conn)
    print(f"\n * {table.upper()} TABLE:")
    print(df)



 * CUSTOMERS TABLE:
   customer_id       name                  email
0            1  Customer1  customer1@example.com
1            2  Customer2  customer2@example.com
2            3  Customer3  customer3@example.com
3            4  Customer4  customer4@example.com
4            5  Customer5  customer5@example.com

 * ORDERS TABLE:
   order_id  customer_id  order_date
0         1            1  2024-11-28
1         2            6  2024-12-02
2         3           14  2025-01-08
3         4           11  2024-09-12
4         5            9  2024-12-24

 * PRODUCTS TABLE:
   product_id product_name     category  price
0           1     Product1  Electronics  47.97
1           2     Product2  Electronics  12.68
2           3     Product3     Clothing  29.68
3           4     Product4        Books  55.48
4           5     Product5        Books  12.39

 * ORDER_ITEMS TABLE:
   order_item_id  order_id  product_id  quantity  price
0              1         1           5         2  12.39
1       

Close the database connection

In [ ]:
conn.close()

Connect to the SQLite database

In [ ]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

#### b) List all customers and any orders they may have placed (customers with no orders will show NULL in order details)

In [ ]:
# Query to list all customers and their orders (if any)
query = """
SELECT 
    c.customer_id, 
    c.name AS customer_name, 
    c.email, 
    o.order_id, 
    o.order_date
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id;
"""

# Execute the query and load the result into a DataFrame
df = pd.read_sql_query(query, conn)

# Display the result
print("\n * b) CUSTOMERS AND THEIR ORDERS:")
print(df)



 * b) CUSTOMERS AND THEIR ORDERS:
    customer_id customer_name                   email  order_id  order_date
0             1     Customer1   customer1@example.com       1.0  2024-11-28
1             2     Customer2   customer2@example.com      17.0  2024-11-25
2             2     Customer2   customer2@example.com      30.0  2024-12-24
3             3     Customer3   customer3@example.com      10.0  2024-09-28
4             3     Customer3   customer3@example.com      22.0  2025-01-05
5             3     Customer3   customer3@example.com      29.0  2024-11-20
6             3     Customer3   customer3@example.com      33.0  2024-10-08
7             4     Customer4   customer4@example.com       9.0  2025-02-12
8             4     Customer4   customer4@example.com      12.0  2025-01-18
9             4     Customer4   customer4@example.com      20.0  2024-09-07
10            4     Customer4   customer4@example.com      35.0  2025-01-07
11            5     Customer5   customer5@example.com

#### c) For each order, list its items with detailed product information

In [ ]:
# Query to list each order with item details and product info
query = """
SELECT 
    o.order_id,
    o.customer_id,
    o.order_date,
    oi.product_id,
    p.product_name,
    p.category,
    p.price AS product_price,
    oi.quantity,
    (p.price * oi.quantity) AS total_cost
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
JOIN products p ON oi.product_id = p.product_id
ORDER BY o.order_id;
"""

# Execute the query and load the result into a DataFrame
df = pd.read_sql_query(query, conn)

# Display the result
print("\n * c) ORDER DETAILS WITH PRODUCT INFORMATION:")
print(df)


 * c) ORDER DETAILS WITH PRODUCT INFORMATION:
    order_id  customer_id  order_date  product_id product_name     category  \
0          1            1  2024-11-28           5     Product5        Books   
1          2            6  2024-12-02           9     Product9  Electronics   
2          3           14  2025-01-08          10    Product10         Toys   
3          3           14  2025-01-08          10    Product10         Toys   
4          4           11  2024-09-12           4     Product4        Books   
..       ...          ...         ...         ...          ...          ...   
65        38           15  2024-11-21           4     Product4        Books   
66        39           12  2024-09-19           1     Product1  Electronics   
67        39           12  2024-09-19           5     Product5        Books   
68        40            6  2024-11-05           7     Product7  Electronics   
69        40            6  2024-11-05           8     Product8         Toys   

    

#### d) Get a complete view of each order including customer details and product details. (all table data)

In [ ]:
# Query to get a complete view of each order with customer & product details
query = """
SELECT 
    o.order_id,
    o.order_date,
    c.customer_id,
    c.name AS customer_name,
    c.email AS customer_email,
    oi.product_id,
    p.product_name,
    p.category,
    p.price AS product_price,
    oi.quantity,
    (p.price * oi.quantity) AS total_cost
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
JOIN products p ON oi.product_id = p.product_id
ORDER BY o.order_id;
"""

# Execute the query and load the result into a DataFrame
df = pd.read_sql_query(query, conn)

# Display the result
print("\n * d) COMPLETE VIEW OF EACH ORDER (CUSTOMER + PRODUCT DETAILS):")
print(df)


 * d) COMPLETE VIEW OF EACH ORDER (CUSTOMER + PRODUCT DETAILS):
    order_id  order_date  customer_id customer_name          customer_email  \
0          1  2024-11-28            1     Customer1   customer1@example.com   
1          2  2024-12-02            6     Customer6   customer6@example.com   
2          3  2025-01-08           14    Customer14  customer14@example.com   
3          3  2025-01-08           14    Customer14  customer14@example.com   
4          4  2024-09-12           11    Customer11  customer11@example.com   
..       ...         ...          ...           ...                     ...   
65        38  2024-11-21           15    Customer15  customer15@example.com   
66        39  2024-09-19           12    Customer12  customer12@example.com   
67        39  2024-09-19           12    Customer12  customer12@example.com   
68        40  2024-11-05            6     Customer6   customer6@example.com   
69        40  2024-11-05            6     Customer6   customer6@ex

#### e) For each customer, count the number of orders and compute the total amount spent.

In [ ]:
# Query to get number of orders and total amount spent per customer
query = """
SELECT 
    c.customer_id,
    c.name AS customer_name,
    COUNT(o.order_id) AS total_orders,
    COALESCE(SUM(p.price * oi.quantity), 0) AS total_amount_spent
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id
LEFT JOIN order_items oi ON o.order_id = oi.order_id
LEFT JOIN products p ON oi.product_id = p.product_id
GROUP BY c.customer_id
ORDER BY total_amount_spent DESC;
"""

# Execute the query and load the result into a DataFrame
df = pd.read_sql_query(query, conn)

# Display the result
print("\n * e) CUSTOMER ORDER COUNT & TOTAL AMOUNT SPENT:")
print(df)


 * e) CUSTOMER ORDER COUNT & TOTAL AMOUNT SPENT:
    customer_id customer_name  total_orders  total_amount_spent
0             7     Customer7             5             1047.27
1             4     Customer4             9              889.59
2            12    Customer12             8              832.95
3             3     Customer3             5              632.67
4             2     Customer2             3              628.20
5             6     Customer6             3              609.73
6             8     Customer8             5              594.19
7            14    Customer14             2              504.24
8            19    Customer19             3              492.91
9             9     Customer9             5              453.33
10           15    Customer15             3              379.81
11           18    Customer18             4              372.28
12           11    Customer11             4              323.56
13           13    Customer13             5           

#### f) For each product, sum up the quantities sold across all orders

In [ ]:
# Query to sum up the total quantity sold per product
query = """
SELECT 
    p.product_id,
    p.product_name,
    p.category,
    SUM(oi.quantity) AS total_quantity_sold
FROM products p
LEFT JOIN order_items oi ON p.product_id = oi.product_id
GROUP BY p.product_id
ORDER BY total_quantity_sold DESC;
"""

# Execute the query and load the result into a DataFrame
df = pd.read_sql_query(query, conn)

# Display the result
print("\n * f) TOTAL QUANTITY SOLD PER PRODUCT:")
print(df)


 * f) TOTAL QUANTITY SOLD PER PRODUCT:
   product_id product_name     category  total_quantity_sold
0           2     Product2  Electronics                   34
1           9     Product9  Electronics                   26
2           7     Product7  Electronics                   25
3           5     Product5        Books                   24
4          10    Product10         Toys                   23
5           8     Product8         Toys                   23
6           4     Product4        Books                   21
7           3     Product3     Clothing                   16
8           1     Product1  Electronics                   14
9           6     Product6        Books                   10


Close the connection:


In [ ]:
conn.close()